# COVID-19 Simulation

This COVID-19 simulation was developed by [transentis consulting](https://www.transentis.com) from Berlin, based on the [SIR model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model).

Please read the companion blog post [Covid 19 and the SIR Model](https://www.transentis.com/covid-19-sir-model/) for details. The simulation was built using the [BPTK-Py](https://bptk.transentis-labs.com) framework, the dasboard using [Voila](https://voila.readthedocs.io/en/stable/). You can find the complete source code on [GitHub](https://github.com/transentis/sim-covid-19). 

In [1]:
from BPTK_Py import Model
from BPTK_Py import sd_functions as sd

model = Model(starttime=1.0,stoptime=1500.0,dt=1.0,name='sir')

susceptible = model.stock("S")
infectious = model.stock("I")
recovered = model.stock("R")
deaths = model.stock("D")

susceptible.initial_value = 80000000.0
infectious.initial_value = 120.0
recovered.initial_value = 0.0
deaths.initial_value = 0.0

infection_rate = model.flow("IR")
recovery_rate = model.flow("RR")
death_rate = model.flow("DR")

contact_rate = model.converter("c")
total_population = model.converter("TP")
net_population = model.converter("N")
intensive_needed = model.converter("needed")
intensive_rate = model.constant("irate")
infectivity = model.constant("i")
intensive_available = model.constant("available")
lethality = model.constant("L")
average_duration = model.constant("d")

susceptible.equation = -infection_rate
infectious.equation = infection_rate - recovery_rate - death_rate
recovered.equation = recovery_rate
deaths.equation = death_rate

infectivity.equation = 0.02
intensive_rate.equation = 0.002
intensive_available.equation = 30000.0
lethality.equation = 0.001
average_duration.equation = 20.0

contact_rate.equation = 20.0 
total_population.equation = susceptible+infectious+recovered
net_population.equation = total_population-deaths
intensive_needed.equation =infectious*intensive_rate

infection_rate.equation = (contact_rate*susceptible*infectivity)*(infectious/total_population)
recovery_rate.equation = infectious/average_duration
death_rate.equation = infectious*lethality

import BPTK_Py
import pandas as pd
bptk = BPTK_Py.bptk()
bptk.register_model(model)

bptk.register_scenarios(
    scenarios ={
        "contactLargeGroups": {},
        "contactSmallGroups": {},
        "contactTwoPeople": {},
        "interactiveScenario":{}
    },
    scenario_manager="smSir")

In [7]:
%run src/dashboard/sir_dashboard.ipy